<a href="https://colab.research.google.com/github/pritiyadav888/Machine-Learning-Projects/blob/main/google_vantilator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [113]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [151]:
# Importing DataScience libraries
import pandas as pd
import gc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm,skew
from scipy import stats
%matplotlib inline
from sklearn.linear_model import LinearRegression,Ridge,RidgeCV, ElasticNetCV, LassoCV,BayesianRidge
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler,StandardScaler,MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error

In [115]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Predictive_Data/vantilator_google/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Predictive_Data/vantilator_google/test.csv')

In [116]:
train.describe()

,id,breath_id,R,C,time_step,u_in,u_out,pressure
count,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06
mean,3.018000e+06,6.283886e+04,2.703618e+01,2.608072e+01,1.307225e+00,7.321615e+00,6.204493e-01,1.122041e+01
std,1.742443e+06,3.633526e+04,1.959549e+01,1.715231e+01,7.659778e-01,1.343470e+01,4.852752e-01,8.109703e+00
min,1.000000e+00,1.000000e+00,5.000000e+00,1.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,-1.895744e+00
25%,1.509001e+06,3.137700e+04,5.000000e+00,1.000000e+01,6.428995e-01,3.936623e-01,0.000000e+00,6.329607e+00
50%,3.018000e+06,6.276550e+04,2.000000e+01,2.000000e+01,1.308123e+00,4.386146e+00,1.000000e+00,7.032628e+00
75%,4.527000e+06,9.430100e+04,5.000000e+01,5.000000e+01,1.965502e+00,4.983895e+00,1.000000e+00,1.364103e+01
max,6.036000e+06,1.257490e+05,5.000000e+01,5.000000e+01,2.937238e+00,1.000000e+02,1.000000e+00,6.482099e+01


In [117]:
train.isnull().sum().max()

0

In [118]:
train = train.fillna(0)
test = test.fillna(0)

In [119]:
train_df = train.head(10000)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         10000 non-null  int64  
 1   breath_id  10000 non-null  int64  
 2   R          10000 non-null  int64  
 3   C          10000 non-null  int64  
 4   time_step  10000 non-null  float64
 5   u_in       10000 non-null  float64
 6   u_out      10000 non-null  int64  
 7   pressure   10000 non-null  float64
dtypes: float64(3), int64(5)
memory usage: 625.1 KB


In [120]:
test_df = test.head(10000)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         10000 non-null  int64  
 1   breath_id  10000 non-null  int64  
 2   R          10000 non-null  int64  
 3   C          10000 non-null  int64  
 4   time_step  10000 non-null  float64
 5   u_in       10000 non-null  float64
 6   u_out      10000 non-null  int64  
dtypes: float64(2), int64(5)
memory usage: 547.0 KB


In [121]:
# X array - drop target
X= train_df.drop(['pressure'],axis=1).values

In [122]:
# y array - target only
y=train_df['pressure'].values

In [123]:
# Split Train into Train and Validate Sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20,random_state=42)

In [124]:
# Size of our newly split datasets 
print('Train size: %i' % X_train.shape[0])
print('Validation size: %i' % X_val.shape[0])

Train size: 8000
Validation size: 2000


In [125]:
# No. of folds for Cross Validation
n_folds = 5
# Defining a function which returns the mean squared error between the cross validation score of X and y arrays
def rmse_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42)
    rmse= np.sqrt(-cross_val_score(model,X_train, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [126]:
# Intialize the Cross Validation function with Shuffle
n_folds = 5
custom_cv = KFold(n_folds, shuffle=True, random_state=42)

In [127]:
#define  our  GB model
GBR = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [128]:
#fitting he model on the train set
GBR.fit(X_train,y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.05, loss='huber',
                          max_depth=4, max_features='sqrt', max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=15, min_samples_split=10,
                          min_weight_fraction_leaf=0.0, n_estimators=3000,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=5, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [129]:
#Cross Validation Error
rmse_cv(GBR).mean()

2.207325944250817

In [130]:
# R squared, RMSE and Mean Absolute Error obtained by using actual and predicted values from Validate set
print('Validation R^2: %.5f'  % r2_score(y_val,GBR.predict(X_val)))
print('Validation RMSE: %.5f\n'   % mean_squared_error(y_val,GBR.predict(X_val)))
print('Validation Mean Absolute Error: %.5f\n'   % mean_absolute_error(y_val,GBR.predict(X_val)))

Validation R^2: 0.94670
Validation RMSE: 3.96578

Validation Mean Absolute Error: 0.98730



In [131]:
# ridgecv model
ridgecv = RidgeCV(alphas=[0.1, 1.0, 10.0],cv =5)

In [132]:
#fitting our model on the training data
ridgecv.fit(X_train,y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]), cv=5, fit_intercept=True,
        gcv_mode=None, normalize=False, scoring=None, store_cv_values=False)

In [133]:
# let's call our function to calculate the mean cross validation error
rmse_cv(ridgecv).mean()

6.502206031889818

In [134]:
# R- squared, RMSE and Mean Absolute Error for the validation sets
print('Validation R^2: %.5f'  % r2_score(y_val,ridgecv.predict(X_val)))
print('Validation RMSE: %.5f\n'   % mean_squared_error(y_val,ridgecv.predict(X_val)))
print('Validation Mean Absolute Error: %.5f\n'   % mean_absolute_error(y_val,ridgecv.predict(X_val)))

Validation R^2: 0.41054
Validation RMSE: 43.85605

Validation Mean Absolute Error: 4.11785



In [135]:
# kernel model
KRR = KernelRidge(alpha=5)

In [136]:
# fitting on train
KRR.fit(X_train,y_train)

KernelRidge(alpha=5, coef0=1, degree=3, gamma=None, kernel='linear',
            kernel_params=None)

In [137]:
#  model estimatores
print('Validation R^2: %.5f'  % r2_score(y_val,KRR.predict(X_val)))
print('Validation RMSE: %.5f\n'   % mean_squared_error(y_val,KRR.predict(X_val)))
print('Validation Mean Absolute Error: %.5f\n'   % mean_absolute_error(y_val,KRR.predict(X_val)))

Validation R^2: 0.18946
Validation RMSE: 60.30473

Validation Mean Absolute Error: 5.43725



In [138]:
# define the model with alpha values in a numpy array with cross-validation function included
lasso = LassoCV(alphas = [1, 0.1, 0.001, 0.0005],cv =custom_cv)

In [139]:
# fit the model
lasso.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 74418.31997747056, tolerance: 44.15761982165143
  tol, rng, random, positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 129313.32525384909, tolerance: 44.15761982165143
  tol, rng, random, positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 28792.151532791468, tolerance: 44.15761982165143
  tol, rng, random, positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to incr

LassoCV(alphas=[1, 0.1, 0.001, 0.0005], copy_X=True,
        cv=KFold(n_splits=5, random_state=42, shuffle=True), eps=0.001,
        fit_intercept=True, max_iter=1000, n_alphas=100, n_jobs=None,
        normalize=False, positive=False, precompute='auto', random_state=None,
        selection='cyclic', tol=0.0001, verbose=False)

In [140]:
# Model scoring
print('Validation R^2: %.5f'  % r2_score(y_val,lasso.predict(X_val)))
print('Validation RMSE: %.5f\n'   % mean_squared_error(y_val,lasso.predict(X_val)))
print('Validation Mean Absolute Error: %.5f\n'   % mean_absolute_error(y_val,lasso.predict(X_val)))

Validation R^2: 0.41054
Validation RMSE: 43.85602

Validation Mean Absolute Error: 4.11793



In [141]:
#CV error
rmse_cv(lasso).mean()

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 61150.44317998258, tolerance: 35.1048569845247
  tol, rng, random, positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 101119.16217682742, tolerance: 35.1048569845247
  tol, rng, random, positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22939.717004101898, tolerance: 35.1048569845247
  tol, rng, random, positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increas

6.502211573009637

In [142]:
# defining the model with alphas and Elasticnet mixing parameter l1_ratio
Elastic = ElasticNetCV(alphas=[0.0001, 0.0005, 0.001, 0.01, 0.1, 1, 10], l1_ratio=[.01, .1, .5, .9, .99], max_iter=5000
                       ,cv=custom_cv)

In [143]:
# fit the model
Elastic.fit(X_train,y_train)

ElasticNetCV(alphas=[0.0001, 0.0005, 0.001, 0.01, 0.1, 1, 10], copy_X=True,
             cv=KFold(n_splits=5, random_state=42, shuffle=True), eps=0.001,
             fit_intercept=True, l1_ratio=[0.01, 0.1, 0.5, 0.9, 0.99],
             max_iter=5000, n_alphas=100, n_jobs=None, normalize=False,
             positive=False, precompute='auto', random_state=None,
             selection='cyclic', tol=0.0001, verbose=0)

In [144]:
# Model scores
print('Validation R^2: %.5f'  % r2_score(y_val,Elastic.predict(X_val)))
print('Validation RMSE: %.5f\n'   % mean_squared_error(y_val,Elastic.predict(X_val)))
print('Validation Mean Absolute Error: %.5f\n'   % mean_absolute_error(y_val,Elastic.predict(X_val)))

Validation R^2: 0.41055
Validation RMSE: 43.85572

Validation Mean Absolute Error: 4.11783



In [145]:
# CV error
rmse_cv(Elastic).mean()

6.502256497220327

In [146]:
# import pipeline
from sklearn.pipeline import Pipeline
# initialize pipeline with the model and here we have function which takes cares of outlier
GBRPipe = Pipeline([
        ('outlier', RobustScaler()),
        ('gbm', GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5))])

In [147]:
# set up the parameter grid with parameters you like to iterate on. Here we have 3
param_grid = {'gbm__learning_rate': [0.01, 0.05],
             'gbm__max_depth': [1,5],
             'gbm__min_samples_leaf': [10, 15]}

In [148]:
#  setup gridsearchcv with estimator, param_grid
GBRGrid = GridSearchCV(estimator = GBRPipe,param_grid=param_grid)

In [149]:
GBRGrid.fit(X_train,y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('outlier',
                                        RobustScaler(copy=True,
                                                     quantile_range=(25.0,
                                                                     75.0),
                                                     with_centering=True,
                                                     with_scaling=True)),
                                       ('gbm',
                                        GradientBoostingRegressor(alpha=0.9,
                                                                  ccp_alpha=0.0,
                                                                  criterion='friedman_mse',
                                                                  init=None,
                                                                  learning_rate=0.05,
                                          

In [152]:
gc.collect()

452

In [154]:
print('Validation R^2: %.5f'  % r2_score(y_val,GBRGrid.predict(X_val)))
print('Validation RMSE: %.5f\n'   % mean_squared_error(y_val,GBRGrid.predict(X_val)))
print('Validation Mean Absolute Error: %.5f\n'   % mean_absolute_error(y_val,GBRGrid.predict(X_val)))

Validation R^2: 0.95762
Validation RMSE: 3.15340

Validation Mean Absolute Error: 0.85230



In [155]:
GBRGrid.predict(test)

array([7.81016349, 7.54148497, 7.75877013, ..., 5.90048401, 5.89371883,
       5.82004007])

In [156]:
GBRGrid.best_score_

0.9466034074895132

In [157]:
predicted = GBRGrid.predict(test)
predicted

array([7.81016349, 7.54148497, 7.75877013, ..., 5.90048401, 5.89371883,
       5.82004007])

In [158]:
test_result = test
test_result['pressure'] = predicted

In [159]:
submit_result = test_result[['id','pressure']]
submit_result

,id,pressure
0,1,7.810163
1,2,7.541485
2,3,7.758770
3,4,8.771505
4,5,10.524447
...,...,...
4023995,4023996,5.842115
4023996,4023997,5.876811
4023997,4023998,5.900484
4023998,4023999,5.893719


In [160]:
submit_result.to_csv('/content/drive/MyDrive/Colab Notebooks/Predictive_Data/vantilator_google/submission.csv', index=False)